In [1]:
!pip install OpenAI google-api-python-client
!pip install langchain pyowm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType,ZeroShotAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory


import os

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENWEATHERMAP_API_KEY"] = ""

llm= OpenAI(temperature =0, model_name = 'gpt-3.5-turbo')

tools = load_tools(["openweathermap-api",'google-search'], llm)


prefix = """Yor are a weather activity application, and you are trained to return the weather of the location. If ask any further subsequent question for the location, you need to answer the question considering the location given by user as a input. To better utitlize the tools, you should save the country of the location. Whenever using the OpenWeatherMap tool, you should pass the actuion input in this form, City/State/location, Country name, eg. Texas,US. You have access to the following tools"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)



In [4]:
agent_chain.run("I am in Brooklyn, What should I wear today between 5Pm -6Pm?")




> Entering new AgentExecutor chain...
Thought: I need to know the weather for this location
Action: OpenWeatherMap
Action Input: Brooklyn, US
Observation: In Brooklyn, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 5.66 m/s, direction: 120°
Humidity: 55%
Temperature: 
  - Current: 19.15°C
  - High: 23.7°C
  - Low: 16.03°C
  - Feels like: 18.56°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: I now know the weather for this location
Final Answer: Since it is currently 19.15°C and the wind speed is 5.66 m/s, you should wear a light jacket or a sweater.

> Finished chain.


'Since it is currently 19.15°C and the wind speed is 5.66 m/s, you should wear a light jacket or a sweater.'

In [5]:
agent_chain.run("What games should I play today?")




> Entering new AgentExecutor chain...
Thought: I should look for something related to the current weather.
Action: OpenWeatherMap
Action Input: Brooklyn, US
Observation: In Brooklyn, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 5.66 m/s, direction: 120°
Humidity: 55%
Temperature: 
  - Current: 19.15°C
  - High: 23.7°C
  - Low: 16.03°C
  - Feels like: 18.56°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: I should look for outdoor activities that can be done in this weather.
Action: Google Search
Action Input: Outdoor activities in Brooklyn
Observation: These experiences are best for outdoor activities in Brooklyn: New York Aquarium Admission Ticket · The Empire Helicopter Tour of New York · Lower Manhattan and ... Mar 13, 2023 ... There's no shortage of outdoor activities in New York, ... favorites such as the Brooklyn Flea, LIC Flea and Food and Queens Night Market. Top 10 Best Outdoor Activities Nearby in Brooklyn, New York · Citrovia · Gotham Arche

'Based on the current weather in Brooklyn, some outdoor activities you can enjoy today include visiting the New York Aquarium, taking an Empire Helicopter Tour of New York, visiting the Brooklyn Flea, LIC Flea and Food, visiting the Queens Night Market, visiting Luna Park in Coney Island, visiting Prospect Park Zoo, visiting the Brooklyn Botanical Garden, visiting the Brooklyn Heights Promenade, visiting the Gingerbread House of Bay Ridge, sipping wine at a rooftop vineyard, getting your sweat on by the pool, eating oysters on the river, picnicking in Prospect Park, enjoying a beach day at the Prospect Park Tour, visiting the Brooklyn Botanic Garden, visiting the Brooklyn Bridge Park, visiting the Salt Marsh Nature Center, camping in Brooklyn, visiting Coney Island Beach and Boardwalk, and visiting BRIC Celebrate Brooklyn!'

In [6]:
agent_chain.run('Should I apply sun screen cream before going outside?')



> Entering new AgentExecutor chain...
Thought: What is the current weather?
Action: OpenWeatherMap
Action Input: Brooklyn, US
Observation: In Brooklyn, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 5.66 m/s, direction: 120°
Humidity: 55%
Temperature: 
  - Current: 19.15°C
  - High: 23.7°C
  - Low: 16.03°C
  - Feels like: 18.56°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: The temperature is quite high and there is no cloud cover, so it is likely to be sunny.
Action: Google Search
Action Input: Should I apply sun screen cream before going outside?
Observation: Your skin can take up to a half-hour to absorb sunscreen. So, plan ahead and apply sunscreen 30 minutes before going outside. And, don't skimp on the ... Regular daily use of SPF 15 sunscreen can reduce your risk of developing squamous cell carcinoma ... WHEN: Every day; 30 minutes prior to going outdoors. To protect your lips, apply a lip balm with a SPF of at least 15. To remain protected whe

'Yes, you should apply sunscreen cream 30 minutes before going outside to protect your skin from the sun.'

In [7]:
agent_chain.run('Which events can I attend between 5Pm-6PM today?')



> Entering new AgentExecutor chain...
Thought: I should use Google Search to find events happening in Brooklyn.
Action: Google Search
Action Input: Events in Brooklyn
Observation: Brooklyn events in New York, NY · Brooklyn Wine Fest · BROOKLYN NIGHT MARKET · Brooklyn Job Fair June 29, 2023 - Brooklyn Career Fairs · The Market at the Brooklyn ... Post Your Event · Back to Brooklyn Paper; Discover. All Events · Animals · Art · Books & Readings · Caribbean · Classes · Community · Craft & DIY · Dance ... Join the Brooklyn Bridge Park Conservancy at four annual parties in support of free events and activities in Brooklyn Bridge Park. Events. Downtown Brooklyn Presents, Music, Dance, Free. Capicu! The Party: Uptown Vinyl Supreme. A celebration of Nuyorican Soul through Dominoes, ... Bonsai Bar events pop up all over the city at locations like Brooklyn Brewery, the Bronx Brewery and SingleCut Beersmiths Queens Taproom. Read more · Buy ticket. Discover the best things to do & events in Brook

'Some of the events you can attend between 5PM-6PM today in Brooklyn include Brooklyn Wine Fest, BROOKLYN NIGHT MARKET, Brooklyn Job Fair June 29, 2023 - Brooklyn Career Fairs, The Market at the Brooklyn, Downtown Brooklyn Presents, Music, Dance, Free, Capicu! The Party: Uptown Vinyl Supreme, Patina Events at Brooklyn Botanic Garden, Bonsai Bar events, BAM (Brooklyn Academy of Music), Ace Hotel Brooklyn events, and BPL Presents programs at the Dr. S. Stevan Dweck Cultural Center at Central Library.'

In [8]:
agent_chain.run('What are the best place to eat for vegetarians?')



> Entering new AgentExecutor chain...
Thought: I need to find a place to eat for vegetarians
Action: Google Search
Action Input: "best places to eat for vegetarians in Brooklyn"
Observation: May 9, 2023 ... For good reason: The Indian food truck specializing in all things crunchy dosas is one of the city's most affordable (and portable) vegan ... Jul 31, 2022 ... The Top 10 Plant-Based and Vegan Restaurants in Brooklyn, New York · 4. Bunna Cafe. Calling all: Vegans trying to stretch their dollar! · 6. VSpot. Best Vegetarian Friendly Restaurant near me in Brooklyn, New York · Daughter · Nura · Recette · The Brooklyn Tree · Lakou Cafe · Hart's Restaurant · Dinner Party. Jan 13, 2023 ... 27. Modern Love Brooklyn ... Vegan comfort food is the focus of this Williamsburg charmer: On the menu, find soul warmers like seitan buffalo ... Vegetarian Friendly Restaurants in Brooklyn‎ · 1. Sottocasa Pizzeria. (357). Closed Now. Italian, Pizza$$ - $$$ Menu · 2. Oregano. (305). Closed Now · 3. Kuu 

"There are many great vegetarian and vegan restaurants in Brooklyn, such as LuAnne's Wild Ginger, Ital Kitchen, Bunna Cafe, Champs Diner, Little Choc Apothecary, Jungle Cafe, Hartbreakers, Modern Love Brooklyn, Aunts et Uncles, and Sottocasa Pizzeria."

In [9]:
agent_chain.run("Is it cloudy today?")



> Entering new AgentExecutor chain...
Thought: I need to check the weather
Action: OpenWeatherMap
Action Input: Brooklyn, US
Observation: In Brooklyn, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 5.66 m/s, direction: 120°
Humidity: 55%
Temperature: 
  - Current: 19.15°C
  - High: 23.7°C
  - Low: 16.03°C
  - Feels like: 18.56°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: I now know the final answer
Final Answer: No, it is not cloudy today in Brooklyn, US.

> Finished chain.


'No, it is not cloudy today in Brooklyn, US.'

In [10]:
agent_chain.run('When should I go outside in the Manhattan?')



> Entering new AgentExecutor chain...
Thought: I need to know the weather conditions in Manhattan
Action: OpenWeatherMap
Action Input: Manhattan, US
Observation: In Manhattan, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 4.63 m/s, direction: 50°
Humidity: 52%
Temperature: 
  - Current: 19.83°C
  - High: 23.56°C
  - Low: 15.9°C
  - Feels like: 19.23°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: I now know the weather conditions in Manhattan
Final Answer: The best time to go outside in Manhattan is when the temperature is comfortable and the wind speed is low.

> Finished chain.


'The best time to go outside in Manhattan is when the temperature is comfortable and the wind speed is low.'

In [11]:
agent_chain.run('Should I apply sun screen cream before going outside?')



> Entering new AgentExecutor chain...
Thought: I need to know the weather conditions
Action: OpenWeatherMap
Action Input: Brooklyn, US
Observation: In Brooklyn, US, the current weather is as follows:
Detailed status: smoke
Wind speed: 5.66 m/s, direction: 120°
Humidity: 55%
Temperature: 
  - Current: 19.15°C
  - High: 23.7°C
  - Low: 16.03°C
  - Feels like: 18.56°C
Rain: {}
Heat index: None
Cloud cover: 0%
Thought: The temperature is not too hot and the wind speed is low, so sunscreen is recommended
Final Answer: Yes, you should apply sunscreen cream 30 minutes before going outside to protect your skin from the sun.

> Finished chain.


'Yes, you should apply sunscreen cream 30 minutes before going outside to protect your skin from the sun.'

In [12]:
agent_chain.run('What are the best flight deals to Miami from my location?')



> Entering new AgentExecutor chain...
Thought: I need to find flight deals
Action: Google Search
Action Input: Best flight deals to Miami from Brooklyn
Observation: ... on Spirit Airlines from $37... Search for Miami flights on KAYAK now to find the best deal. ... Top tips for finding a cheap flight out of New York. Find cheap flights from Miami International to New York. Search and compare millions of airline tickets to find cheap flight deals. Top tips for finding a cheap flight out of Miami · The cheapest flight from Miami to New York was found 43 days before departure, on average. · Book at least 2 ... Book one-way or return flights from New York to Miami with no change fee on selected flights. Earn double with airline miles + Expedia Rewards points. Changes to Pay with Miles tickets are determined by the fare rules of the ticket purchased. Basic Economy | Basic Economy fares are non-refundable, do not ... Book one-way or return flights from Miami to New York with no change fee o

'The best flight deals to Miami from Brooklyn can be found on Spirit Airlines from $37, and other deals can be found on KAYAK, JetBlue Airways, and Expedia.'

In [13]:
agent_chain.run('what is the nearest airport?')



> Entering new AgentExecutor chain...
Thought: I need to find the nearest airport
Action: Google Search
Action Input: nearest airport to Brooklyn
Observation: There is no major airport in Brooklyn,it is instead served by New York's airports. We recommend you fly to John F Kennedy Intl Airport, however you can also ... Major airports near Brooklyn, New York: ; 12 miles · John F. Kennedy International Airport (JFK / KJFK) ; 12 miles · LaGuardia Airport (LGA / KLGA) ; 23 miles ... Nearest Airport to Brooklyn, NY, United States. The closest airport to Brooklyn is New York John F. Kennedy International Airport (JFK). Closest airports to Brooklyn ... The nearest airport to Brooklyn is New York JFK (JFK). MTA operates a vehicle from 14 St-Union Sq to Church Av every 20 minutes. Your best bet is likely JFK, as you can take the Airlink to Jamaica Station, and then take the LIRR to Flatbush Avenue. There are tons of subway lines at ... Nearest airports to the LIU Brooklyn campus are John F. Ke

'The nearest airport to Brooklyn is John F. Kennedy International Airport (JFK).'

In [14]:
agent_chain.run('what is the average time to reach to Time square?')



> Entering new AgentExecutor chain...
Thought: I need to know the distance between Brooklyn and Time Square
Action: Google Search
Action Input: Distance between Brooklyn and Time Square
Observation: May 9, 2023 ... There are 3 ways to get from Times Square to Brooklyn by subway, taxi or car ; Subway · 27 min. Take the subway from Times Sq-42 St to Prospect ... To walk from Times Square to the Brooklyn end of the Brooklyn Bridge is about five miles -- with one mile just being the Bridge itself. Where does the Brooklyn to Times Square train arrive? Brooklyn to Times Square train services, operated by MTA, arrive at Times Sq-42 St station. · Can I drive ... Sadly, a lot of first-time visitors spend a disproportionate amount of their limited time in New York around Times Square – whether it's for eating, shopping, ... We were stopping in Times Square too! Clarke Street was the subway stop we went to in Brooklyn - the bridge is signposted from the station. I think it took ... Margaritavil

'The average time to reach Time Square from Brooklyn is approximately 38 minutes by subway, with a fare of $2.75.'